In [5]:
# %%bash
import os
os.environ['http_proxy'] = 'http://localhost:8081'
os.environ['https_proxy'] = 'http://localhost:8081'
os.environ['NCCL_P2P_DISABLE'] = '1'
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [6]:
%cd '/mnt/nfs-data/users/anhvth5/projects/guidance/'

/mnt/nfs-data/users/anhvth5/projects/guidance


In [7]:
from speedy import *
from guidance import *

In [8]:
data = load_by_ext('../tvpl_crawler/data/tvpl_qa_short_answer_giaothong/qa_short_answer/*')
questions = []
for qa in data:
    questions.extend(qa['questions'])

2023-12-30 11:22:00.025 | INFO     | speedy.utils:multi_process:262 - Multi processing load_by_ext | Num samples: 6788


  0%|          | 0/6788 [00:00<?, ?it/s]

In [9]:
# llm = models.get_kilm_guidance('../kilm-finetuning/output/kilm_chat_2m_samples/', device_map=0, do_update_lm_head=True, echo=False)
# llm = models.get_qwen_guidance('../kilm-finetuning/output/Qwen-1_8B-Chat_on_giaothong_rag/', device_map=0, do_update_lm_head=True, echo=True)
llm72 = models.get_qwen_guidance("/public-llm/Qwen-14B-Chat-Int4/",device_map=0, do_update_lm_head=True, echo=True)

2023-12-30 11:22:18.230 | INFO     | guidance.models.transformers._qwen:get_qwen_guidance:227 - Load Qwen from "/public-llm/Qwen-14B-Chat-Int4/" with device_map "0"


Using `disable_exllama` is deprecated and will be removed in version 4.37. Use `use_exllama` instead and specify the version with `exllama_config`.The value of `use_exllama` will be overwritten by `disable_exllama` passed in `GPTQConfig` or stored in your config file.
You passed `quantization_config` to `from_pretrained` but the model you're loading already has a `quantization_config` attribute and has already quantized weights. However, loading attributes (e.g. use_exllama, exllama_config, use_cuda_fp16, max_input_length) will be overwritten with the one you passed to `from_pretrained`. The rest will be ignored.
Try importing flash-attention for faster inference...


Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

2023-12-30 11:22:39.945 | INFO     | guidance.models.transformers._qwen:__init__:47 - Set tokenizer.eos_token_id = tokenizer.eod_id


len(tokenizer)=151851


In [10]:
import requests

def search(query, k=3, port=19001):
    url = f"http://localhost:{port}/api/search"
    params = {
        "query": query,
        "return_dict": "true",
        'k': k
    }
    response = requests.get(url, params=params, proxies={'http':'', 'https':''})
    topk = response.json()['topk']
    # print(topk[0].keys())
    # docid_to_text = {t['pid']:f'{t["title"]}| {t["text"]}' for t in topk}
    docid_to_text = [{'doc_id': t['pid'], 'text': f'{t["title"]}| {t["text"]}'} for t in topk]
    return docid_to_text
query = 'Thủ tục đăng ký biển số xe máy mới nhất'
search_results = search(query, k=3)
print(json.dumps(search_results, ensure_ascii=False, indent=2))

[
  {
    "doc_id": 30668,
    "text": "Thông tư 24/2023/TT-BCA cấp thu hồi đăng ký biển số xe cơ giới mới nhất\nChương II\nMục 2. ĐĂNG KÝ SANG TÊN, DI CHUYỂN XE |  Điều 15. Thủ tục đăng ký sang tên, di chuyển xe\n1. Thủ tục thu hồi\na) Chủ xe kê khai giấy khai thu hồi đăng ký, biển số xe trên cổng dịch vụ công; cung cấp mã hồ sơ đăng ký xe trực tuyến; nộp hồ sơ thu hồi quy định tại khoản 1 Điều 14 Thông tư này và nhận giấy hẹn trả kết quả đăng ký xe theo quy định;\nb) Sau khi kiểm tra hồ sơ xe hợp lệ, cơ quan đăng ký xe cấp chứng nhận thu hồi đăng ký, biển số xe theo quy định (có dán bản chà số máy, số khung và đóng dấu giáp lai của cơ quan đăng ký xe trên bản chà số máy, số khung xe): 01 bản trả cho chủ xe; 01 bản lưu hồ sơ xe; trường hợp mất chứng nhận đăng ký xe thì thực hiện xác minh theo quy định."
  },
  {
    "doc_id": 121823,
    "text": "Thông tư 24/2023/TT-BCA cấp thu hồi đăng ký biển số xe cơ giới mới nhất\nChương II\nMục 2. ĐĂNG KÝ SANG TÊN, DI CHUYỂN XE |  Điều 15. Thủ tụ

In [11]:
import guidance

In [14]:
@guidance
def giaothong_rag(lm, query, gt_answer=None, k=None, db="qa"):
    port = 19003 if db == "qa" else 19001
    if k is None:
        # elase randomly choose int form 5 to 10
        k = 3 if db == "qa" else np.random.randint(3, 7)
    relevant_docs = search(query, k=k, port=port)

    with system():
        lm = (
            lm
            + "You are now tasked to answer user questions based on the retrieved documents\nResponse language: Vietnamese"
        )

    with user():
        lm += """Query: {}
        Retrieved documents: {}
        Guide:
            - Hãy phân tích sau đó đưa ra kết luận về câu hỏi này
            - Response language: Vietnamese
        """.format(
            query, json.dumps(relevant_docs, ensure_ascii=False, indent=2)
        )

    with assistant():
        lm += f"**Câu hỏi** {query}"
        lm += "\n\n**Phân tích**" + gen(max_tokens=2000, name="reasoning", stop="**")
        lm += "\n\n**Kết luận**" + gen(
            max_tokens=2000, name="conclusion", stop="<|im_end|>"
        )
    with user():
        lm += "Hãy trích dẫn nguồn tham khảo cho kết luận trên, nếu không có hãy trả lời `Không có tài liệu liên quan` \n\
        Khi bạn trích dẫn nguồn, hãy liệt kê theo thứ tự ưu tiên. Tài liệu quan trọng nhất nên được đặt ở vị trí đầu tiên. \n\
        Cách định dạng: ```reference[{'doc_id': 'document_id', 'point': điểm liên quan)}, {doc_id_2:điểm liên quan}, ...]``` \n\
        Điểm liên quan được tính trên thang điểm 10 với 10 là điểm liên quan nhất, 0 là không liên quan."
    with assistant():
        lm += select(['Không có tài liệu liên quan', 'Các tài liệu liên quan'], name="has_reference")
        if lm['has_reference'] == 'Các tài liệu liên quan':
            lm += "\n ```reference["
            lm += gen(max_tokens=2000, name="reference", stop="]```")
            lm += "]```"
        

    return lm


In [15]:
qa = questions[0]
lm = llm72+giaothong_rag(query = qa['question'], db='')